<a href="https://colab.research.google.com/github/IANSEZI/machinelearning/blob/main/Model007GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
!pip install scikit-learn
!pip install numpy

In [2]:
import tensorflow as tf
import cv2
import numpy as np
from google.colab import drive
import os

drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Function to load and preprocess images for a specific split (training/testing)
def load_images_and_labels(split_folder, image_size):
    images = []
    labels = []

    # Iterate over each class folder
    for class_name in os.listdir(split_folder):
        class_folder = os.path.join(split_folder, class_name)
        if os.path.isdir(class_folder):
            # Iterate over images in the class folder
            for image_name in os.listdir(class_folder):
                image_path = os.path.join(class_folder, image_name)
                # Load and preprocess image
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
                image = cv2.resize(image, (image_size, image_size))  # Resize image
                # Append image and label
                images.append(image)
                labels.append(class_name)

    # Convert lists to NumPy arrays
    images = np.array(images)
    labels = np.array(labels)

    return images, labels

# Set image size
image_size = 48  # Adjust as needed

# Load training images and labels
training_folder = '/content/drive/My Drive/faces/train'  # Adjust the path to your training folder
X_train, y_train = load_images_and_labels(training_folder, image_size)

# Load testing images and labels
testing_folder = '/content/drive/My Drive/faces/test'  # Adjust the path to your testing folder
X_test, y_test = load_images_and_labels(testing_folder, image_size)

# Shuffle the training data
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Display some information about the dataset
print("Number of training images:", len(X_train))
print("Number of testing images:", len(X_test))
print("Image shape:", X_train[0].shape)
print("Classes:", np.unique(y_train))


Number of training images: 6530
Number of testing images: 1423
Image shape: (48, 48)
Classes: ['Angry' 'Disgust' 'Fear' 'Happy' 'Neutral' 'Sad' 'Surprise']


In [5]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Encode class labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert encoded labels to one-hot encoding
num_classes = len(np.unique(y_train))
y_train_onehot = to_categorical(y_train_encoded, num_classes=num_classes)
y_test_onehot = to_categorical(y_test_encoded, num_classes=num_classes)

# Normalize pixel values of images to range [0, 1]
X_train_normalized = X_train.astype('float32') / 255.0
X_test_normalized = X_test.astype('float32') / 255.0

# Display the number of classes
print("Number of classes:", num_classes)

Number of classes: 7


In [6]:
from tensorflow.keras import layers, models

# Define the GCN model
def create_gcn_model(input_shape, num_classes):
    model = models.Sequential()

    # Graph convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())

    # Fully connected layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Define input shape
input_shape = (X_train_normalized.shape[1], X_train_normalized.shape[2], 1)

# Create the GCN model
gcn_model = create_gcn_model(input_shape, num_classes)

# Compile the model
gcn_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# Display model summary
gcn_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 4, 4, 128)         0

In [7]:
# Train the model
batch_size = 32
epochs = 20

history = gcn_model.fit(X_train_normalized, y_train_onehot,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(X_test_normalized, y_test_onehot))

# Evaluate the model on the testing data
loss, accuracy = gcn_model.evaluate(X_test_normalized, y_test_onehot)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/20
205/205 [==============================] - 25s 113ms/step - loss: 1.9237 - accuracy: 0.1637 - val_loss: 1.8922 - val_accuracy: 0.2284
Epoch 2/20
205/205 [==============================] - 24s 118ms/step - loss: 1.8544 - accuracy: 0.2441 - val_loss: 1.8029 - val_accuracy: 0.2755
Epoch 3/20
205/205 [==============================] - 22s 108ms/step - loss: 1.7726 - accuracy: 0.2772 - val_loss: 1.7354 - val_accuracy: 0.3071
Epoch 4/20
205/205 [==============================] - 24s 117ms/step - loss: 1.6969 - accuracy: 0.3230 - val_loss: 1.6789 - val_accuracy: 0.3373
Epoch 5/20
205/205 [==============================] - 22s 109ms/step - loss: 1.6283 - accuracy: 0.3703 - val_loss: 1.5717 - val_accuracy: 0.3879
Epoch 6/20
205/205 [==============================] - 24s 115ms/step - loss: 1.5622 - accuracy: 0.3965 - val_loss: 1.5219 - val_accuracy: 0.4132
Epoch 7/20
205/205 [==============================] - 24s 119ms/step - loss: 1.4723 - accuracy: 0.4407 - val_loss: 1.4822 - val_ac

In [8]:
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from sklearn.metrics import classification_report

In [14]:
# Train the model
batch_size = 40
epochs = 20

history = gcn_model.fit(X_train_normalized, y_train_onehot,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(X_test_normalized, y_test_onehot))

# Evaluate the model on the testing data
loss, accuracy = gcn_model.evaluate(X_test_normalized, y_test_onehot)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)



Epoch 1/20
164/164 [==============================] - 23s 138ms/step - loss: 0.6355 - accuracy: 0.7665 - val_loss: 1.7609 - val_accuracy: 0.4793
Epoch 2/20
164/164 [==============================] - 24s 145ms/step - loss: 0.6036 - accuracy: 0.7735 - val_loss: 1.7408 - val_accuracy: 0.4884
Epoch 3/20
164/164 [==============================] - 22s 133ms/step - loss: 0.5657 - accuracy: 0.7855 - val_loss: 1.8899 - val_accuracy: 0.4828
Epoch 4/20
164/164 [==============================] - 24s 147ms/step - loss: 0.5336 - accuracy: 0.7972 - val_loss: 1.8890 - val_accuracy: 0.4807
Epoch 5/20
164/164 [==============================] - 23s 138ms/step - loss: 0.4979 - accuracy: 0.8135 - val_loss: 1.9791 - val_accuracy: 0.4779
Epoch 6/20
164/164 [==============================] - 24s 146ms/step - loss: 0.4581 - accuracy: 0.8265 - val_loss: 2.0372 - val_accuracy: 0.4736
Epoch 7/20
164/164 [==============================] - 24s 144ms/step - loss: 0.4457 - accuracy: 0.8314 - val_loss: 2.1814 - val_ac

NameError: name 'y_true_labels' is not defined

In [19]:
from sklearn.metrics import classification_report
y_true_labels = ['a', 'b', 'c', 'd','e','f','g','h']
y_pred_labels = ['a', 'b', 'b', 'd','e','f','g','h']

print(classification_report(y_true_labels, y_pred_labels))


              precision    recall  f1-score   support

           a       1.00      1.00      1.00         1
           b       0.50      1.00      0.67         1
           c       0.00      0.00      0.00         1
           d       1.00      1.00      1.00         1
           e       1.00      1.00      1.00         1
           f       1.00      1.00      1.00         1
           g       1.00      1.00      1.00         1
           h       1.00      1.00      1.00         1

    accuracy                           0.88         8
   macro avg       0.81      0.88      0.83         8
weighted avg       0.81      0.88      0.83         8



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
print(X_train.shape)

(6530, 48, 48)


In [10]:
print(input_shape)

(48, 48, 1)


In [11]:
total_elements_X_train = X_train.size
print(f"Total elements in X_train: {total_elements_X_train}")

Total elements in X_train: 15045120


In [12]:
total_elements_desired_shape = 2240 * 48 * 48 * 3
print(f"Total elements in desired shape: {total_elements_desired_shape}")

Total elements in desired shape: 15482880


In [13]:
if total_elements_X_train != total_elements_desired_shape:
    print("Error: The total number of elements in X_train is not compatible with the desired shape.")

Error: The total number of elements in X_train is not compatible with the desired shape.
